In [11]:
pip install pytelegrambotapi

     |████████████████████████████████| 71kB 2.1MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-3.7.0-cp36-none-any.whl size=50347 sha256=80dc2dbf83dbc63b6cf7291171854641171ccc554cc06a7fc00f428913f4cbfe
  Stored in directory: /root/.cache/pip/wheels/48/d5/5b/61aaed368818013983556c076a7d3132b33cea219811a1a6eb
Successfully built pytelegrambotapi


In [0]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout



classifier = Sequential()

#Adding Conv layers to NN
classifier.add(Conv2D(48,(3,3), input_shape = (86, 86, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (3, 3)))

classifier.add(Conv2D(48,(3,3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (3, 3)))

classifier.add(Flatten())

#Adding fully connected layers
classifier.add(Dense(units = 92, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(units = 32, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [7]:
classifier.load_weights('bclassifier.h5')
print('loaded model weights')

loaded model weights


In [9]:
#Checking model perfomance by photo from internet
import numpy as np
from keras.preprocessing import image

photo = 'teen1'
test_image = image.load_img('/content/%s.jpg' % photo, target_size = (86,86))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
answer1 = classifier.predict(test_image)


if answer1[0][0] == 0:
    print('old')
    history[photo] = 'old'
else:
    print('young')

    
    history[photo] = 'young'

         #for 'old' = 0, 'young = 1

young


In [0]:
#Training the model
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory('/content/drive/My Drive/train/train/train_set',
                                                 target_size = (86, 86),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('/content/drive/My Drive/data/train',
                                            target_size = (86, 86),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         steps_per_epoch = 625,
                         epochs = 4,
                         validation_data = test_set,
                         validation_steps = 10)


history = {}

#Checking model perfomance by photo from internet
import numpy as np
from keras.preprocessing import image

photo = 'teen1'
test_image = image.load_img('/content/%s.jpg' % photo, target_size = (86,86))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
answer1 = classifier.predict(test_image)


if answer1[0][0] == 0:
    print('old')
    history[photo] = 'old'
else:
    print('young')

    
    history[photo] = 'young'

         #for 'old' = 0, 'young = 1

In [48]:
#telegram bot code 
#run localy here the Username @ProjectML_bot


import numpy as np
from keras.preprocessing import image
import telebot
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
import numpy as np
from keras.preprocessing import image
from pathlib import Path

import os



classifier = Sequential()

#Adding Conv layers to NN
classifier.add(Conv2D(48,(3,3), input_shape = (86, 86, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (3, 3)))

classifier.add(Conv2D(48,(3,3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (3, 3)))

classifier.add(Flatten())

#Adding fully connected layers
classifier.add(Dense(units = 92, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(units = 32, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

classifier.load_weights('bclassifier.h5')
print('loaded model weights')

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
bot = telebot.TeleBot('1123439556:AAEyXXE9NMmYfxB-WVi4XHKN-zzcVsJ7PIE')
@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Hi do you want to know are you old or young /start')

@bot.message_handler(content_types=['photo'])
def handle_docs_photo(message):
 file_info = bot.get_file(message.photo[len(message.photo)-1].file_id)
 downloaded_file = bot.download_file(file_info.file_path)
 src='/content/'+file_info.file_path;
 with open(src, 'wb') as new_file:
  new_file.write(downloaded_file)

    
  directory = '/content/photos'
  files = os.listdir(directory)
  images = filter(lambda x: x.endswith('.jpg'), files)
  photo = next(images)
  test_image = image.load_img('/content/photos/%s' % photo, target_size = (86,86))
  test_image = image.img_to_array(test_image)
  test_image = np.expand_dims(test_image, axis = 0)
  answer1 = classifier.predict(test_image)


  if answer1[0][0] == 0:
      bot.reply_to(message, "old")
      print('old')
      for f in Path('/content/photos').glob('*.jpg'):
        try:
            f.unlink()
        except OSError as e:
            print("Error: %s : %s" % (f, e.strerror))
      
      #file_path = '/content/photos/file_0.jpg'
      #os.remove(file_path)   

      history[photo] = 'old' 
  else:
      bot.reply_to(message, "young")
      print('young')
      for f in Path('/content/photos').glob('*.jpg'):
        try:
            f.unlink()
        except OSError as e:
            print("Error: %s : %s" % (f, e.strerror))
      
      
      
      #file_path = '/content/photos/file_0.jpg'
      #os.remove(file_path)  

    
      history[photo] = 'young'

         #for 'old' = 0, 'young = 1
   
   
 
bot.polling()

loaded model weights


2020-04-29 20:45:57,983 (util.py:66 PollingThread) ERROR - TeleBot: "ApiException occurred, args=('A request to the Telegram API was unsuccessful. The server returned HTTP 409 Conflict. Response body:\n[b\'{"ok":false,"error_code":409,"description":"Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"}\']',)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/telebot/util.py", line 60, in run
    task(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/telebot/__init__.py", line 281, in __retrieve_updates
    updates = self.get_updates(offset=(self.last_update_id + 1), timeout=timeout)
  File "/usr/local/lib/python3.6/dist-packages/telebot/__init__.py", line 251, in get_updates
    json_updates = apihelper.get_updates(self.token, offset, limit, timeout, allowed_updates)
  File "/usr/local/lib/python3.6/dist-packages/telebot/apihelper.py", line 199, in get_updates
    return _make_request(token, meth

young
young
young
young
old
old
old
old
old
young
old
young
young
old
young
